# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd
import numpy as np

URL = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
mkt = pd.read_csv(URL)

mkt.columns = (mkt.columns
               .str.strip()
               .str.lower()
               .str.replace(r'[^0-9a-zA-Z]+', '_', regex=True)
               .str.strip('_'))


for col in ["customer_lifetime_value", "income", "monthly_premium_auto", "total_claim_amount", "number_of_policies"]:
    if col in mkt.columns:
        mkt[col] = pd.to_numeric(mkt[col], errors="coerce")


if "effective_to_date" in mkt.columns:
    mkt["effective_to_date"] = pd.to_datetime(mkt["effective_to_date"], errors="coerce")
    mkt["month"] = mkt["effective_to_date"].dt.to_period("M").astype(str)


for c in ["state","policy_type","gender","education","sales_channel","response","coverage"]:
    if c in mkt.columns and mkt[c].dtype == "object":
        mkt[c] = mkt[c].astype(str).str.strip().str.title()

print("Shape:", mkt.shape)
mkt.head(3)


Shape: (10910, 27)


/var/folders/cv/pfmqj_4n6fg_b8vt_pytwxwr0000gn/T/ipykernel_26389/942043714.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  mkt["effective_to_date"] = pd.to_datetime(mkt["effective_to_date"], errors="coerce")


,unnamed_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN,2011-02
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN,2011-01
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2011-02


1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [ ]:

CLAIM_THRESHOLD = 1000


print(mkt.columns.tolist())


if mkt["total_claim_amount"].dtype == "object":
    mkt["total_claim_amount"] = pd.to_numeric(mkt["total_claim_amount"], errors="coerce")


mkt["response"] = mkt["response"].astype(str).str.strip().str.lower()


mask = (mkt["total_claim_amount"] < CLAIM_THRESHOLD) & (mkt["response"] == "yes")


low_claim_yes = mkt.loc[mask].copy()

print("Filtered rows:", low_claim_yes.shape[0])
low_claim_yes.head()


['unnamed_0', 'customer', 'state', 'customer_lifetime_value', 'response', 'coverage', 'education', 'effective_to_date', 'employmentstatus', 'gender', 'income', 'location_code', 'marital_status', 'monthly_premium_auto', 'months_since_last_claim', 'months_since_policy_inception', 'number_of_open_complaints', 'number_of_policies', 'policy_type', 'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size', 'vehicle_type', 'month']
Filtered rows: 1399


,unnamed_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
3,3,XL78013,Oregon,22332.439460,yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,2011-01
8,8,FM55990,California,5989.773931,yes,Premium,College,2011-01-19,Employed,M,...,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,NaN,2011-01
15,15,CW49887,California,4626.801093,yes,Basic,Master,2011-01-16,Employed,F,...,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,NaN,2011-01
19,19,NJ54277,California,3746.751625,yes,Extended,College,2011-02-26,Employed,F,...,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A,2011-02
27,27,MQ68407,Oregon,4376.363592,yes,Premium,Bachelor,2011-02-28,Employed,F,...,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,NaN,2011-02


2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [ ]:

mkt["response"] = mkt["response"].astype(str).str.strip().str.lower()


responded = mkt.loc[mkt["response"] == "yes"].copy()

print("Customers who responded 'Yes':", responded.shape[0])
responded.head()


Customers who responded 'Yes': 1466


,unnamed_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
3,3,XL78013,Oregon,22332.439460,yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,2011-01
8,8,FM55990,California,5989.773931,yes,Premium,College,2011-01-19,Employed,M,...,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,NaN,2011-01
15,15,CW49887,California,4626.801093,yes,Basic,Master,2011-01-16,Employed,F,...,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,NaN,2011-01
19,19,NJ54277,California,3746.751625,yes,Extended,College,2011-02-26,Employed,F,...,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A,2011-02
27,27,MQ68407,Oregon,4376.363592,yes,Premium,Bachelor,2011-02-28,Employed,F,...,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,NaN,2011-02


In [ ]:

summary = responded.groupby(["policy_type", "gender"], dropna=False)[
    ["monthly_premium_auto", "customer_lifetime_value"]
].mean().round(2)

print("Average Monthly Premium & CLV by Policy Type and Gender:")
display(summary)


Average Monthly Premium & CLV by Policy Type and Gender:


monthly_premium_auto  customer_lifetime_value
policy_type    gender                                               
Corporate Auto F                      94.30                  7712.63
               M                      92.19                  7944.47
Personal Auto  F                      99.00                  8339.79
               M                      91.09                  7448.38
Special Auto   F                      92.31                  7691.58
               M                      86.34                  8247.09

In [5]:
claim_summary = responded.groupby(["policy_type", "gender"], dropna=False)[
    "total_claim_amount"
].mean().round(2)

print("Average Total Claim Amount by Policy Type and Gender:")
display(claim_summary)


Average Total Claim Amount by Policy Type and Gender:


policy_type     gender
Corporate Auto  F         433.74
                M         408.58
Personal Auto   F         452.97
                M         457.01
Special Auto    F         453.28
                M         429.53
Name: total_claim_amount, dtype: float64

In [6]:
combined_summary = (
    responded.groupby(["policy_type", "gender"], dropna=False)
    .agg({
        "monthly_premium_auto": "mean",
        "customer_lifetime_value": "mean",
        "total_claim_amount": "mean"
    })
    .round(2)
    .rename(columns={
        "monthly_premium_auto": "avg_monthly_premium",
        "customer_lifetime_value": "avg_clv",
        "total_claim_amount": "avg_total_claim"
    })
)

display(combined_summary)


avg_monthly_premium  avg_clv  avg_total_claim
policy_type    gender                                               
Corporate Auto F                     94.30  7712.63           433.74
               M                     92.19  7944.47           408.58
Personal Auto  F                     99.00  8339.79           452.97
               M                     91.09  7448.38           457.01
Special Auto   F                     92.31  7691.58           453.28
               M                     86.34  8247.09           429.53

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [ ]:

customers_by_state = mkt["state"].value_counts()

print("Number of customers per state:")
display(customers_by_state)


Number of customers per state:


state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Nan            631
Name: count, dtype: int64

In [8]:

large_states = customers_by_state[customers_by_state > 500]

print("States with more than 500 customers:")
display(large_states)


States with more than 500 customers:


state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Nan            631
Name: count, dtype: int64

In [9]:
large_states_df = large_states.reset_index()
large_states_df.columns = ["state", "num_customers"]
display(large_states_df.sort_values("num_customers", ascending=False))


,state,num_customers
0,California,3552
1,Oregon,2909
2,Arizona,1937
3,Nevada,993
4,Washington,888
5,Nan,631


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:

clv_stats = (
    mkt.groupby(["education", "gender"], dropna=False)["customer_lifetime_value"]
       .agg(["max", "min", "median"])
       .round(2)
       .sort_values(by="median", ascending=False)
)

print("Customer Lifetime Value (CLV) statistics by education and gender:")
display(clv_stats)
#conclusion

Customer Lifetime Value (CLV) statistics by education and gender:


max      min   median
education            gender                            
High School Or Below M       83325.38  1940.98  6286.73
                     F       55277.45  2144.92  6039.55
College              M       61134.68  1918.12  6005.85
Master               F       51016.07  2417.78  5729.86
Bachelor             F       73225.96  1904.00  5640.51
College              F       61850.19  1898.68  5623.61
Master               M       50568.26  2272.31  5579.10
Doctor               M       32677.34  2267.60  5577.67
Bachelor             M       67907.27  1898.01  5548.03
Doctor               F       44856.11  2395.57  5332.46

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here